# Análisis de discurso
Instalación de API de Anthropic para el análisis de datos con la tecnología de Sonnet (específicamente claude-3-sonnet-20240229)

Lo primero es tomar la información de la clave secreta como variable de entorno a través de un input que protege la información privada de la API KEY

In [ ]:
# Instalar las bibliotecas necesarias
%pip install anthropic

: 

In [ ]:
# Importar las bibliotecas necesarias
import os
from anthropic import Client
from IPython.display import display, HTML
from google.colab import output

# Interfaz para ingresar la API key
def request_anthropic_api_key():
    input_html = """
    
        Proveer API Key
        Por favor, ingrese su API key de Anthropics en el siguiente campo:
        
        
    
    
    """
    display(HTML(input_html))

# Callback para almacenar la API key en la variable de entorno
def store_anthropic_api_key(api_key):
    os.environ["ANTHROPIC_API_KEY"] = api_key
    print("API key almacenada correctamente en la variable de entorno.")

# Registrar la función en Google Colab
output.register_callback('notebook.store_anthropic_api_key', store_anthropic_api_key)

# Solicitar la API key al usuario
request_anthropic_api_key()

# Crear un cliente para la API
client = Client(api_key=store_anthropic_api_key)

: 

In [ ]:
from anthropic import Anthropic
import os

def analyze_cft_policy(law_text, ministry_docs):
    """
    Analiza la ley de CFTs estatales y documentos ministeriales relacionados.
    """
    client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

    system_prompt = "Analiza la Ley 20.910 sobre CFTs estatales y su implementación según documentos ministeriales."

    user_message = f"""
    Analiza los siguientes documentos:

    LEY 20.910: {law_text}
    DOCUMENTOS MINISTERIALES: {ministry_docs}
    
    Realiza un análisis considerando:
    1. Marco legal y objetivos establecidos en Ley 20.910
    2. Avances en implementación según Plan de Fortalecimiento
    3. Cumplimiento de hitos y metas
    4. Financiamiento y recursos asignados
    5. Articulación territorial
    6. Convenios y colaboraciones
    7. Infraestructura y equipamiento
    
    Estructura obligatoria:
    - Análisis del marco normativo
    - Estado actual de implementación por región
    - Cumplimiento de objetivos legales
    - Evaluación del plan de fortalecimiento
    - Brechas y desafíos pendientes
    
    Para cada sección:
    - Citar artículos específicos de la ley
    - Contrastar con datos ministeriales
    - Identificar cumplimientos e incumplimientos
    """

    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system_prompt,
        messages=[
            {"role": "user", "content": user_message}
        ]
    )

    return response.content[0].text

def process_documents():
    # Cargar documentos
    with open("ley20910.txt", "r") as f:
        law_text = f.read()
    
    with open("documentos_mineduc.txt", "r") as f:
        ministry_docs = f.read()
    
    # Realizar análisis
    analysis = analyze_cft_policy(law_text, ministry_docs)
    
    # Guardar resultados
    with open("analisis_normativa_cft.txt", "w") as f:
        f.write(analysis)
    
    print("Análisis normativo completado")

if __name__ == "__main__":
    process_documents()

: 